In [1]:
import numpy as np

from pymatgen import Molecule
from pymatgen.core.operations import SymmOp
from pymatgen.util.coord_utils import get_angle

In [2]:
class Polymer(object):
    def __init__(self, monomer, n_units, head, tail, link_distance=1.0):
        """
        Args:
            monomer (Molecule): The monomer
            n_units (int): number of monomer units
            head (int): index of the atom in the monomer that forms the head
            tail (int): tail atom index. monomers will be connected from head to tail
            link_distance (float): distance between consecutive monomers
        """
        self.monomer = monomer
        self.n_units = n_units
        self.start = head
        self.end = tail
        self.link_distance = link_distance
        monomer.translate_sites(range(len(monomer)), -monomer.cart_coords[0])
        self.mon_vector = monomer.cart_coords[tail] - monomer.cart_coords[head]
        self.moves = {1:[1,0,0], 2:[0,1,0], 3:[0,0,1], 4:[-1,0,0], 5:[0,-1,0], 6:[0,0,-1]}
        self.nmoves = len(self.moves.keys())
        self.prev_move = 1
        self.molecule = self.monomer.copy()
        self.length = 1
        self._create()
    
    def _create(self):
        """
        create the polymer from the monomer by random walk
        """
        while self.length != self.n_units:
            self._add_monomer()
            
    def _next_move_direction(self):
        """
        pick a move at random from the list of moves
        """
        move = np.random.randint(1, self.nmoves+1)
        while self.prev_move == (move + 3)%6:
            move = np.random.randint(1, self.nmoves+1)
        self.prev_move = move
        print "move", move
        return np.array(self.moves[move])
    
    def _align_monomer(self, monomer, move_direction):
        """
        rotate the monomer so that it is aligned along the move direction
        """
        axis = np.cross(self.mon_vector, move_direction)
        origin = monomer[self.start].coords
        angle = get_angle(self.mon_vector, move_direction)
        #print "axis, origin, angle", axis, origin, angle
        op = SymmOp.from_origin_axis_angle(origin, axis, angle)
        monomer.apply_operation(op)

    def _add_monomer(self):
        """
        extend the polymer molecule by adding a monomer
        """
        monomer = self.monomer.copy()
        move_direction = self._next_move_direction()
        translate_by = self.molecule.cart_coords[self.end] + self.link_distance * move_direction
        monomer.translate_sites(range(len(monomer)), translate_by)
        self._align_monomer(monomer, move_direction)
        # add monomer if there are no crossings
        does_cross = False
        for i, site in enumerate(monomer):
            try:
                self.molecule.append(site.specie, site.coords)
            except:
                does_cross = True
                polymer_length = len(self.molecule)
                self.molecule.remove_sites(range(polymer_length-i, polymer_length))
                break
        if not does_cross:
            self.length += 1
            self.end += len(self.monomer)
            print "length: ", self.length

In [3]:
peo = Molecule.from_file("PEOmonomer_withH.xyz")
monomer = peo
n_units = 100
head_atom = 0
tail_atom = 5
link_distance = 3.0

# create the polymer
peo_polymer = Polymer(monomer, n_units, head_atom, tail_atom, link_distance)

In [4]:
# write the polymer molecule
peo_polymer.molecule.to(filename="polymer.xyz", fmt="xyz")

Use openbabel to optimize the polymer chain.
Local optimization is used to preserve the chain structure.

In [5]:
from pymatgen.io.babel import BabelMolAdaptor

peo_obabel = BabelMolAdaptor(peo_polymer.molecule)
# ['gaff', 'ghemical', 'mmff94', 'mmff94s', 'uff']
peo_obabel.localopt(forcefield='mmff94', steps=500)
peo_optimized = peo_obabel.pymatgen_mol
peo_optimized.to(filename="polymer_optimized.xyz", fmt="xyz")

Use Packmol to pack the optimized polymer chains into a box

In [6]:
from rubicon.io.packmol.packmol import PackmolRunner 

packmol_config = [{"number": 1, "fixed": [0, 0, 0, 0, 0, 0],"centerofmass": ""},
                  {"number": 10, "inside sphere": [0, 0, 0, 200.0]}]
pmr = PackmolRunner([peo_optimized, peo_optimized],
                    packmol_config,
                    tolerance=2.0,
                    filetype="xyz",
                    control_params={"nloop": 1000},
                    auto_box=False, output_file="poly_packed.xyz")
packed_polymer = pmr.run()

packed molecule written to poly_packed.xyz


Get the polymer mixture topology: atoms, bonds, angles and dihedrals

In [7]:
import openbabel

peo_obabel = BabelMolAdaptor(packed_polymer)

obmol = peo_obabel.openbabel_mol

atoms = [x.GetIdx()-1 for x in openbabel.OBMolAtomIter(obmol)]
bonds = [[x.GetBeginAtomIdx()-1, x.GetEndAtomIdx()-1] 
         for x in openbabel.OBMolBondIter(obmol)] 
angles = [list(x) for x in openbabel.OBMolAngleIter(obmol)]
dihedrals = [list(x) for x in openbabel.OBMolTorsionIter(obmol)]

print len(atoms), len(bonds), len(angles), len(dihedrals)

9900 8613 13552 5478
